# Triple A - Group Project
## Predicting Taxi Demand in spatial and time resolution

In this workbook we combine all available data to get combined datasets for different temporal (1h, 2h, 6h, 12h) and spatial (Community Area, Census Tract) resolutions.
Later, we will consider using h3 Uber Hexagon mapping across the city of Chicago top show why using a finer resolution in our eyes is not helpful with this specific data and use case.

In [1]:
import pandas as pd 
import ast
import h3
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Polygon
import holidays
import osmnx as ox
import gc
import folium

import matplotlib.pyplot as plt

In [2]:
ca_to_ct = pd.read_csv("data/CensusTractsTIGER2010_20250711.csv", encoding='latin-1')
ca_to_ct = ca_to_ct.rename(columns={'GEOID10': 'Tract', 'COMMAREA':'CommunityAreaNumber'})

trips_data = pd.read_csv("data/Taxi_Trips__2024-__20250505.csv", parse_dates = ['Trip Start Timestamp'])
chi_weather = pd.read_csv("data/chicago_weather.csv")
poi = pd.read_csv("data/CHI_POI.csv")

C:\Users\LIWFG\AppData\Local\Temp\ipykernel_24400\4178553867.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trips_data = pd.read_csv("data/Taxi_Trips__2024-__20250505.csv", parse_dates = ['Trip Start Timestamp'])


#### Loading of Taxi Trips Data
Since we consider demand, we will have to drop NAs of Pickp Community Areas. If we want to consider the finer resolution of Census Tracts, we have the option to <br>
A) drop all rows missing the actual Census Tract or <br>
B) infer a uniform distribution among Census Tracts from the available Community Area. <br>
 

In [3]:
print(f'Total Observations: {trips_data.shape[0]}')
print(f'Dropping {trips_data['Pickup Community Area'].isna().sum()} Amount of Rows due to NaN Pickup Community Area')
trips_data = trips_data.dropna(subset=['Pickup Community Area'])
print(f'{trips_data.shape[0]} Observations left')

trips_data['Trip Start Timestamp'] = pd.to_datetime(trips_data['Trip Start Timestamp'])
trips_data['Trip Hour'] = trips_data['Trip Start Timestamp'].dt.floor('h')

Total Observations: 7917844
Dropping 226777 Amount of Rows due to NaN Pickup Community Area
7691067 Observations left


Before we do this, we need to check if the assignment of tracts to Cummunity Areas is right for all existing Tracts in our Trips Data. If so, we can infer the other Tracts to be assigned correctly in an informed manner.

In [4]:
trips = trips_data.copy()
mapping = ca_to_ct.copy()

trips = trips[trips['Pickup Census Tract'].notna()].copy()
mapping['CommunityAreaNumber'] = mapping['CommunityAreaNumber'].astype(int)

trips['Pickup Census Tract'] = trips['Pickup Census Tract'].astype(int)
trips['Pickup Community Area'] = trips['Pickup Community Area'].astype(int)

# Find any pickup tracts in trips_data not in the mapping
pickup_tracts = set(trips['Pickup Census Tract'].unique())
mapped_tracts = set(mapping['Tract'].unique())

missing_pickup = sorted(pickup_tracts - mapped_tracts)
print("Pickup tracts missing in ca_to_ct mapping:", missing_pickup)

# For the ones that *are* in the mapping, check CA match
pickup_check = (
    trips[['Trip ID', 'Pickup Census Tract', 'Pickup Community Area']]
    .rename(columns={
        'Pickup Census Tract': 'Tract',
        'Pickup Community Area': 'trips_CA'
    })
    .merge(
        mapping[['Tract', 'CommunityAreaNumber']].rename(
            columns={'CommunityAreaNumber': 'map_CA'}
        ),
        on='Tract',
        how='left'
    )
)
pickup_check['match'] = pickup_check['trips_CA'] == pickup_check['map_CA']

mismatches = pickup_check[~pickup_check['match']]
print(f"Found {len(mismatches)} pickup records where the CA doesn’t match:")
print(mismatches.head())

# Clean up
del mapping, trips, mismatches
gc.collect()

Pickup tracts missing in ca_to_ct mapping: []
Found 0 pickup records where the CA doesn’t match:
Empty DataFrame
Columns: [Trip ID, Tract, trips_CA, map_CA, match]
Index: []


20

In [5]:
trips = trips_data.copy()
mapping = ca_to_ct.copy()

mapping['CommunityAreaNumber'] = mapping['CommunityAreaNumber'].astype(int)
trips['Pickup Community Area'] = trips['Pickup Community Area'].astype(int)

# Count tracts per community area
tract_counts = (
    mapping
    .groupby('CommunityAreaNumber')['Tract']
    .count()
    .rename('tract_count')
    .reset_index()
)
mapping = mapping.merge(tract_counts, on='CommunityAreaNumber')

# Split known vs. unknown pickup tracts
known = trips[trips['Pickup Census Tract'].notna()].copy()
known['Tract'] = known['Pickup Census Tract'].astype(int)
known['weight'] = 1.0

unknown = trips[trips['Pickup Census Tract'].isna()].copy()
unknown = unknown.merge(
    mapping[['CommunityAreaNumber', 'Tract', 'tract_count']],
    left_on='Pickup Community Area',
    right_on='CommunityAreaNumber',
    how='left'
)
unknown['weight'] = 1.0 / unknown['tract_count']

# Concatenate and aggregate weighted counts
expanded = pd.concat([
    known[['Trip Hour', 'Tract', 'weight']],
    unknown[['Trip Hour', 'Tract', 'weight']]
], ignore_index=True)

counts_by_hour_tract = (
    expanded
    .groupby(['Trip Hour', 'Tract'])['weight']
    .sum()
    .reset_index(name='trip_count')
)

# Build full cartesian index of all hours × all tracts
all_hours = pd.date_range(
    start=expanded['Trip Hour'].min(),
    end=expanded['Trip Hour'].max(),
    freq='H'
)
all_tracts = mapping['Tract'].unique()

# Create MultiIndex and DataFrame
idx = pd.MultiIndex.from_product(
    [all_hours, all_tracts],
    names=['Trip Hour', 'Tract']
)
full_index = pd.DataFrame(index=idx).reset_index()

# Join the actual counts, fill missing as zero
census_hourly  = (
    full_index
    .merge(counts_by_hour_tract, on=['Trip Hour', 'Tract'], how='left')
)
census_hourly['trip_count'] = census_hourly['trip_count'].fillna(0)

# Clean up
del mapping, trips
gc.collect()

C:\Users\LIWFG\AppData\Local\Temp\ipykernel_24400\2774956827.py:45: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  all_hours = pd.date_range(


0

To get the exact locational data, we have to enrich our dataframe with the Centroids Location from the original data. Since we disaggregate the missing Census Tracts from Community Areas, we have to calculate additional Centroids for the remaining NA coordinates.

In [6]:
matching = trips_data.copy()

matching = matching.rename(columns={'Pickup Census Tract': 'Tract'})
merged_df = census_hourly.merge(
    matching[['Tract', 'Pickup Centroid Location']].drop_duplicates(),
    on='Tract',
    how='left'
)

del matching
gc.collect()

0

In [7]:
if ca_to_ct['the_geom'].dtype == 'O':
    ca_to_ct['geometry'] = ca_to_ct['the_geom'].apply(wkt.loads)
else:
    ca_to_ct['geometry'] = ca_to_ct['the_geom']

gdf = gpd.GeoDataFrame(ca_to_ct, geometry='geometry')


gdf['centroid_point'] = gdf.geometry.centroid
gdf['centroid_wkt'] = gdf['centroid_point'].apply(lambda p: p.wkt)


tract_centroid_wkt = dict(zip(gdf['Tract'], gdf['centroid_wkt']))

mask_na = merged_df['Pickup Centroid Location'].isna()
merged_df.loc[mask_na, 'Pickup Centroid Location'] = merged_df.loc[mask_na, 'Tract'].map(tract_centroid_wkt)


Since we have all Lat/Lon data now, we want to infer h3 hexagons to capture the spatial information between the Tracts.

In [8]:
def extract_lat_lon(point_wkt):
    """
    Takes WKT string like 'POINT (-87.6300448953 41.7424875717)'
    Returns (lat, lon) as floats.
    """
    try:
        # Remove 'POINT (' and ')'
        coords = point_wkt.replace('POINT (', '').replace(')', '').split()
        lon, lat = map(float, coords)
        return lat, lon
    except:
        return None, None

def get_h3_index(row, res=9):
    if pd.isnull(row['Pickup Centroid Location']):
        return None
    lat, lon = extract_lat_lon(row['Pickup Centroid Location'])
    if lat is not None and lon is not None:
        return h3.latlng_to_cell(lat, lon, res)
    else:
        return None


merged_df['pickup_h3_9'] = merged_df.apply(get_h3_index, axis=1)

In [9]:
census_hourly = merged_df.copy()

del merged_df
gc.collect()

0

### Adding Chicago Weather Data from API

In [10]:
def extract_desc(s):
    try:
        data = ast.literal_eval(s)

        if isinstance(data, list) and data:
            return data[0].get('description')

        elif isinstance(data, dict):
            return data.get('description')

    except (ValueError, SyntaxError):
        pass

    return None

#chi_weather['weather_description'] = chi_weather['weather'].apply(extract_desc) # algo can infer the weather description info from all other available data 
chi_weather['nighttime'] = chi_weather['pod'].apply(lambda x: 1 if x == 'n' else 0) # do we need? with sin/cos portraying algo can learn boundaries itself?
chi_weather['timestamp_local'] = pd.to_datetime(chi_weather['timestamp_local'])
chi_weather = chi_weather.drop(columns = ['weather', 'timestamp_utc', 'ts', 
                                            'datetime', 'date', 'slp', 
                                            'dhi', 'dni', 'ghi', 
                                            'solar_rad', 'azimuth', 'elev_angle', 
                                            'h_angle', 'revision_status', 'pod']) # dropping unnecessary columns, can think of also dropping temp since app_temp likely more influential
chi_weather[['clouds', 'pres', 'rh', 'vis', 'wind_dir', 'nighttime']] = chi_weather[['clouds', 'pres', 'rh', 'vis', 'wind_dir', 'nighttime']].astype(float)

#chi_weather.head(5)

We have two missing timestamps ['2024-03-10 02:00:00','2025-03-09 02:00:00'] for which we infer weather data from the previous and following timestamps.

In [11]:
missing_ts = pd.to_datetime(['2024-03-10 02:00', '2025-03-09 02:00'])
chi_weather = chi_weather.set_index('timestamp_local').sort_index()

filled_rows = []
for ts in missing_ts:
    prev_h = ts - pd.Timedelta(hours=1)
    next_h = ts + pd.Timedelta(hours=1)

    row_before = chi_weather.loc[prev_h]
    row_after  = chi_weather.loc[next_h]

    inferred = (row_before + row_after) / 2
    inferred.name = ts
    filled_rows.append(inferred)

df_filled = pd.DataFrame(filled_rows)

chi_weather = pd.concat([chi_weather, df_filled]) \
                .sort_index() \
                .reset_index() \
                .rename(columns={'index':'timestamp_local'})


#print(chi_weather[chi_weather['timestamp_local'] == '2024-03-10 02:00'])
#print(chi_weather[chi_weather['timestamp_local'] == '2025-03-09 02:00'])

In [12]:
weather = chi_weather.copy()
weather['Trip Hour'] = weather['timestamp_local'].dt.floor('H')


weather_cols = [c for c in weather.columns 
                if c not in ('timestamp_local')]

weather_hourly = (
    weather[weather_cols]
    .drop_duplicates(subset='Trip Hour', keep='first')
)

census_hourly = (
    census_hourly
    .merge(weather_hourly, on='Trip Hour', how='left')
)

# Clean up
del weather
gc.collect()

C:\Users\LIWFG\AppData\Local\Temp\ipykernel_24400\1251629888.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  weather['Trip Hour'] = weather['timestamp_local'].dt.floor('H')


21

---
### Match Locational Data from OpenStreetMap
We count amenities (as of now: Restaurants, Cafes, Bars) per Census Tract to add to our Dataframes for prediction. This could be especially interesting when looking at demand in a more granular resolution than Census Tracts and Community Areas.

In [13]:
poi_gdf = poi.copy()
tracts = ca_to_ct.copy()

# parse string into wkt multipolygon object
if poi_gdf.geometry.dtype == object:
    poi_gdf['geometry'] = poi_gdf.geometry.apply(wkt.loads)
poi_gdf = gpd.GeoDataFrame(poi_gdf, geometry='geometry', crs='EPSG:4326')

tracts['geometry'] = tracts['the_geom'].apply(wkt.loads)
tracts = gpd.GeoDataFrame(
    tracts, 
    geometry='geometry', 
    crs='EPSG:4326'
)[['Tract', 'geometry']]

# spatial‐join POIs into tracts
#    “predicate='within'” ensures only points inside the polygon count
joined = gpd.sjoin(
    poi_gdf, 
    tracts, 
    how='inner', 
    predicate='within'
)

poi_counts = (
    joined
    .groupby('Tract')
    .size()
    .reset_index(name='poi_count')
)

# can also count by type (save for later eval)
# poi_counts_by_type = (
#     joined
#     .groupby(['Tract','amenity'])
#     .size()
#     .unstack(fill_value=0)
#     .reset_index()
# )


census_hourly = (
    census_hourly
    .merge(poi_counts, on='Tract', how='left')
)
census_hourly['poi_count'] = census_hourly['poi_count'].fillna(0).astype(int)

#census_hourly.head()

# Clean up
del poi_gdf, tracts, joined, poi_counts
gc.collect()

0

---
### Map public holidays to the final df (+ other date related features)

In [14]:
import holidays

us_holidays = holidays.US(state='IL', years=[2024, 2025])
census_hourly['is_holiday'] = census_hourly['Trip Hour'].dt.date.apply(lambda x: int(x in us_holidays))

#census_hourly.head(5)

# Clean up
del us_holidays
gc.collect()

0

In [15]:
# Weekend Feature
census_hourly['isWeekend'] = (census_hourly['Trip Hour'].dt.dayofweek >= 5).astype(int)

# Day, Hour and Month
census_hourly['Trip Start Day'] = census_hourly['Trip Hour'].dt.day
census_hourly['Trip Start Hour'] = census_hourly['Trip Hour'].dt.hour
census_hourly['Trip Start Month'] = census_hourly['Trip Hour'].dt.month

#census_hourly.head(5)

In [17]:
# we save the df at this point to be able to access without having to run the whole notebook 
census_hourly.to_csv("data/census_hourly_h3.csv", index=False)

---
### Create remaining dfs
We don't do anything new but just sample the hourly data into different temporal formats, summing the trips and using the mean for the remainig data points as the best guessed estimate of weather and all other features.

In [39]:
df = census_hourly.copy()
df_match = df[['pickup_h3_9', 'Tract']].drop_duplicates()
df = df.drop(columns = ['pickup_h3_9', 'Pickup Centroid Location'])

df['Trip Hour'] = pd.to_datetime(df['Trip Hour'])

agg_dict = {
    'trip_count': 'sum',
    **{col: 'mean' for col in df.columns 
          if col not in ['trip_count', 'Tract', 'Trip Hour']}
}

def aggregate_by_freq(df, freq):
    grouped = (
        df.groupby(
            [pd.Grouper(key='Trip Hour', freq=freq), 'Tract'],
            observed=True
        )
        .agg(agg_dict)
        .reset_index()
    )
    merged = grouped.merge(df_match, on='Tract', how='left')
    return merged
    

census_2hourly  = aggregate_by_freq(df, '2H')
census_4hourly  = aggregate_by_freq(df, '4H')
census_6hourly  = aggregate_by_freq(df, '6H')
census_12hourly = aggregate_by_freq(df, '12H')

C:\Users\LIWFG\AppData\Local\Temp\ipykernel_24400\4273584323.py:16: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  [pd.Grouper(key='Trip Hour', freq=freq), 'Tract'],


C:\Users\LIWFG\AppData\Local\Temp\ipykernel_24400\4273584323.py:16: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  [pd.Grouper(key='Trip Hour', freq=freq), 'Tract'],
C:\Users\LIWFG\AppData\Local\Temp\ipykernel_24400\4273584323.py:16: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  [pd.Grouper(key='Trip Hour', freq=freq), 'Tract'],
C:\Users\LIWFG\AppData\Local\Temp\ipykernel_24400\4273584323.py:16: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  [pd.Grouper(key='Trip Hour', freq=freq), 'Tract'],


In [ ]:
census_2hourly.to_csv("data/census_2hourly_h3.csv", index=False)
census_4hourly.to_csv("data/census_4hourly_h3.csv", index=False)
census_6hourly.to_csv("data/census_6hourly_h3.csv", index=False)
census_12hourly.to_csv("data/census_12hourly_h3.csv", index=False)